# Exercise 2 - Vaccindata

In [1]:
# importing all the relevant packages to complete my project
import pandas as pd # Pandas to be able to create and manipulate dataframes
import numpy as np # used for any mathematical functions that may be needed
import seaborn as sns # used for effective plotting with pandas
import plotly_express as px # used for a more interactive plotting
import matplotlib.pyplot as plt # used in conjunction with seaborn

In [3]:
# inporting excel sheet and creating a dataframe from it
df_covid_vaccine = pd.read_excel("../Lab_Covid/Data/Folkhalsomyndigheten_Covid19_Vaccine.xlsx","Vaccinerade kommun och ålder")
df_covid_vaccine.head()

,Län,Län_namn,Kommun,Kommun_namn,Ålder,Befolkning,Antal minst 1 dos,Antal minst 2 doser,Antal 3 doser,Antal 4 doser,Andel minst 1 dos,Andel minst 2 doser,Andel 3 doser,Andel 4 doser
0,1,Stockholms län,114,Upplands Väsby,12-15,2422,1206,1046,NaN,NaN,0.497936,0.431874,NaN,NaN
1,1,Stockholms län,114,Upplands Väsby,16-17,1203,839,755,NaN,NaN,0.697423,0.627598,NaN,NaN
2,1,Stockholms län,114,Upplands Väsby,18-29,6692,4887,4469,1959.0,NaN,0.730275,0.667812,0.292738,NaN
3,1,Stockholms län,114,Upplands Väsby,30-39,7332,5542,5240,2878.0,NaN,0.755865,0.714675,0.392526,NaN
4,1,Stockholms län,114,Upplands Väsby,40-49,6946,5592,5429,3719.0,NaN,0.805068,0.781601,0.535416,NaN


In [7]:
# Will now do some exploitative analysis on the dataset to see what it contains.
df_covid_vaccine.info()
df_covid_vaccine.columns,  df_covid_vaccine.index
# I can see that there is 2900 entries with 14 columns. Some of the columns have NaN values. There is also a mix of string, int and float values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Län                  2900 non-null   int64  
 1   Län_namn             2900 non-null   object 
 2   Kommun               2900 non-null   int64  
 3   Kommun_namn          2900 non-null   object 
 4   Ålder                2900 non-null   object 
 5   Befolkning           2900 non-null   int64  
 6   Antal minst 1 dos    2900 non-null   int64  
 7   Antal minst 2 doser  2900 non-null   int64  
 8   Antal 3 doser        2320 non-null   float64
 9   Antal 4 doser        870 non-null    float64
 10  Andel minst 1 dos    2900 non-null   float64
 11  Andel minst 2 doser  2900 non-null   float64
 12  Andel 3 doser        2320 non-null   float64
 13  Andel 4 doser        870 non-null    float64
dtypes: float64(6), int64(5), object(3)
memory usage: 317.3+ KB


(Index(['Län', 'Län_namn', 'Kommun', 'Kommun_namn', 'Ålder', 'Befolkning',
        'Antal minst 1 dos', 'Antal minst 2 doser', 'Antal 3 doser',
        'Antal 4 doser', 'Andel minst 1 dos', 'Andel minst 2 doser',
        'Andel 3 doser', 'Andel 4 doser'],
       dtype='object'),
 RangeIndex(start=0, stop=2900, step=1))

----
#### a) How many counties(Län) are represented within the dataset

In [15]:
län_count = df_covid_vaccine.groupby("Län_namn").size() # Here i have grouped the column as series to focus on that information
län_count = län_count.size # .size will then count how long the series is. 
print(f"There are {län_count} different Counties represented in this dataset about Covid 19 Vaccinations.") # Printing out my answer as an f string to include the total count of Läns

There are 21 different Counties represented in this dataset about Covid 19 Vaccinations.


----
#### b) How many municipalities are represented within the dataset

In [16]:
kommun_count = df_covid_vaccine.groupby("Kommun_namn").size()
kommun_count = kommun_count.size
print(f"There are {kommun_count} different  in this dataset about Covid 19 Vaccinations")

There are 290 different Counties in this dataset about Covid 19 Vaccinations
